Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [71]:
!pip3 install category_encoders
!pip3 install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from category_encoders.target_encoder import TargetEncoder
from category_encoders.woe import WOEEncoder
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [37]:
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=33)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [0]:
def wrangle(X):
    X = X.copy()

    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    cols_with_zeros = ['longitude', 'latitude', 'amount_tsh', 'construction_year', 'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
            
    cols_high_c = ['ward', 'subvillage', 'funder', 'installer', 'lga', 'wpt_name']

    for col in cols_high_c:
      top10 = X[col].value_counts()[:15].index
      X.loc[~X[col].isin(top10), col] = 'OTHER'

    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()

    X = X.drop(columns=['quantity_group', 'payment', 'recorded_by'])
    
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
target = 'status_group'

# train_features = train.drop(columns=[target, 'id'])

# numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# cardinality = train_features.select_dtypes(exclude='number').nunique()

# categorical_features = cardinality[cardinality <= 100].index.tolist()

# features = numeric_features + categorical_features

In [0]:
X_train = train.drop(columns=[target, 'id'])
y_train = train[target]
X_val = val.drop(columns=[target, 'id'])
y_val = val[target]
X_test = test.drop(columns=['id'])

In [75]:
pipe = make_pipeline(
    OrdinalEncoder(),
    IterativeImputer(random_state=33),
    GradientBoostingClassifier(random_state=33)
)


pipe.fit(X_train, y_train)
accuracy = pipe.score(X_val, y_val)
print(f'Train Accuracy: {pipe.score(X_train, y_train)}\n')
print(f'Validation Accuracy: {accuracy}\n')
# output.eval_js('new Audio("http://noproblo.dayjo.org/ZeldaSounds/LOZ/LOZ_Secret.wav").play()')

Train Accuracy: 0.6923821548821549

Validation Accuracy: 0.6924242424242424



In [26]:
pipe = make_pipeline(
    OrdinalEncoder(),
    IterativeImputer(random_state=33),
    RandomForestClassifier(n_jobs=-1,
                          random_state=33)
)

param_distributions = {
    'iterativeimputer__max_iter': randint(10,500),
    'iterativeimputer__initial_strategy': ['mean', 'median', 'most_frequent'],
    'randomforestclassifier__n_estimators': randint(10, 1000), 
    'randomforestclassifier__max_depth': [randint(5, 200), None], 
    'randomforestclassifier__max_features': [uniform(0, 1), 'sqrt', 'log2', 'auto'],
    'randomforestclassifier__min_samples_leaf': randint(1, 100) 
}

search = RandomizedSearchCV(
    pipe, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

# output.eval_js('new Audio("http://noproblo.dayjo.org/ZeldaSounds/LOZ/LOZ_Secret.wav").play()')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 19

In [65]:
pipe = make_pipeline(
    OrdinalEncoder(),
    IterativeImputer(random_state=33),
    GradientBoostingClassifier(random_state=33)
)

param_distributions = {
    'iterativeimputer__max_iter': randint(10,500),
    'iterativeimputer__initial_strategy': ['mean', 'median', 'most_frequent'],
    'gradientboostingclassifier__n_estimators': randint(10, 1000), 
    'gradientboostingclassifier__max_depth': randint(1, 200), 
    'gradientboostingclassifier__max_features': [None, 'sqrt', 'log2', 'auto'],
    'gradientboostingclassifier__min_samples_leaf': randint(1, 100) 
}

search = RandomizedSearchCV(
    pipe, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-2
)

search.fit(X_train, y_train);

# output.eval_js('new Audio("http://noproblo.dayjo.org/ZeldaSounds/LOZ/LOZ_Secret.wav").play()')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


KeyboardInterrupt: ignored

In [0]:
pipe = search.best_estimator_

In [62]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'gradientboostingclassifier__max_depth': 80, 'gradientboostingclassifier__max_features': 'log2', 'gradientboostingclassifier__min_samples_leaf': 82, 'gradientboostingclassifier__n_estimators': 132, 'iterativeimputer__initial_strategy': 'median', 'iterativeimputer__max_iter': 345}
Cross-validation MAE -0.7978324915824916


In [63]:
accuracy = pipe.score(X_val, y_val)
print(f'Train Accuracy: {pipe.score(X_train, y_train)}\n')
print(f'Validation Accuracy: {accuracy}\n')

Train Accuracy: 0.931986531986532

Validation Accuracy: 0.8115319865319865



In [0]:
submission = test.copy()
submission['status_group'] = pipe.predict(X_test)
submission = submission[['id', 'status_group']]
submission.set_index('id', inplace=True)
submission.head()
submission.to_csv('submission.csv')